In [4]:
%pip install jsonpickle dacite shapely

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import py_planning
py_planning.init()

### Visualize the simulation of planner code

In [2]:
from IPython.display import IFrame
IFrame('http://127.0.0.1:8000', width="100%", height=650)

### Place your planner code here, then run the cell and watch at visualization

In [3]:
from py_planning.data_types import PlannedPath, PlannedState, State
from shapely.geometry import LineString, Point


def get_index_of_closest_point(line: LineString, point: Point):
    closest_point_index = None
    min_distance = float('inf')

    for i, line_point in enumerate(line.coords):
        line_point = Point(line_point)
        distance = point.distance(line_point)
        if distance < min_distance:
            min_distance = distance
            closest_point_index = i

    return closest_point_index


def do_plan(state: State) -> PlannedPath:
    vehicle_pose = state.vehiclePose
    vehicle_pos = Point(vehicle_pose.pos.x, vehicle_pose.pos.y)

    all_centerlines = []
    for centerline in state.lanePath.centerlines:
        all_centerlines.extend(centerline)

    centerline = LineString([(p.x, p.y) for p in all_centerlines])

    closest_index = get_index_of_closest_point(centerline, vehicle_pos)
    current_velocity = vehicle_pose.velocity

    prev_poses_count = 3
    max_poses_count = 50
    first_pose_index = max(closest_index-prev_poses_count, 0)

    planned_states = [
        PlannedState(pos=p, velocity=current_velocity) for p in all_centerlines
    ][first_pose_index:first_pose_index+ max_poses_count]

    #planned_states[3].pos.x += 10

    return PlannedPath(states=planned_states)

py_planning.run_planner(do_plan)

Case failed: 


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58444)
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/lndstol/Dev/dash/py_planning/planning_server.py", line 92, in finish_request
    self.RequestHandlerClass(self.do_plan, self.on_case_status, request, client_address, self)
  File "/Users/lndstol/Dev/dash/py_planning/planning_server.py", line 16, in __init__
    super().__init__(*args, **kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.p